In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Define detector IDs and time range
detector_ids = ["e2w_out", "e2w_in", "w2e_out", "w2e_in"]
start_time = datetime.now().replace(second=0, microsecond=0)
end_time = start_time + timedelta(hours=1)

# Generate random timestamps and detector IDs
data = []
current_time = start_time
while current_time < end_time:
    for detector_id in detector_ids:
        if random.random() > 0.5:  # Randomly decide whether to add an entry
            data.append({"detector_id": detector_id, "timestamp": current_time})
    current_time += timedelta(seconds=random.randint(1, 60))  # Increment time randomly

# Create a DataFrame and save to CSV
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure the column is in datetime format
df['unix_epoch_time'] = (df['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df.reset_index(inplace=True)
df.rename(columns={"index": "id"}, inplace=True)
df['id'] = df['id'].apply(lambda x: str(x)) 
df.to_csv("detector_data.csv", index=False)

print("Random data generated and saved to 'detector_data.csv'.")

Random data generated and saved to 'detector_data.csv'.


In [2]:
df

,id,detector_id,timestamp,unix_epoch_time
0,0,e2w_in,2025-05-09 22:18:00,1746829080
1,1,w2e_in,2025-05-09 22:18:00,1746829080
2,2,e2w_out,2025-05-09 22:18:21,1746829101
3,3,e2w_in,2025-05-09 22:18:21,1746829101
4,4,w2e_in,2025-05-09 22:18:21,1746829101
...,...,...,...,...
237,237,w2e_out,2025-05-09 23:15:27,1746832527
238,238,w2e_in,2025-05-09 23:15:27,1746832527
239,239,w2e_out,2025-05-09 23:16:41,1746832601
240,240,e2w_out,2025-05-09 23:17:41,1746832661


In [3]:
trips = pd.read_csv("detector_data.csv")
trips['depart'] = trips['unix_epoch_time'].copy()
trips['from'] = trips['detector_id'].apply(lambda x: "24225358#0" if x[0]== 'r' else "151884975#0")
trips['to'] = trips['detector_id'].apply(lambda x: "1243253622#0" if x[0]== 'r' else "151884974#0")
trips['departLane'] = trips['detector_id'].apply(lambda x: "0" if x[-1]== 't' else "1")
trips.to_csv("trips_data.csv",index=False)


In [4]:
trips.head()

,id,detector_id,timestamp,unix_epoch_time,depart,from,to,departLane
0,0,e2w_in,2025-05-09 22:18:00,1746829080,1746829080,151884975#0,151884974#0,1
1,1,w2e_in,2025-05-09 22:18:00,1746829080,1746829080,151884975#0,151884974#0,1
2,2,e2w_out,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,0
3,3,e2w_in,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,1
4,4,w2e_in,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,1


In [1]:
import sumolib

# Path to the SUMO network file
network_file = "/home/kaveh/KTH/Hornsgatan/map/Hornsgatan.net"

# Load the network
net = sumolib.net.readNet(network_file)

# Example: Print all edges in the network
for edge in net.getEdges():
    print(f"Edge ID: {edge.getID()}, From: {edge.getFromNode().getID()}, To: {edge.getToNode().getID()}")

Edge ID: -118190636, From: 1879647580, To: 194948
Edge ID: -1285445696, From: 194952, To: 194949
Edge ID: -1303, From: 1647249308, To: 21651854
Edge ID: -151884978#2, From: 8080758, To: 1647249339
Edge ID: -151884978#3, From: 355140447, To: 8080758
Edge ID: -151884978#4, From: 160909, To: 355140447
Edge ID: -1880564#3, From: 8080754, To: 8080755
Edge ID: -1880625#1, From: 21651807, To: 5711961
Edge ID: -24617129#2, From: 1089010847, To: 5537722
Edge ID: -250434485#0, From: 160875, To: 160871
Edge ID: -250434485#1, From: 1639252528, To: 160875
Edge ID: -250434533, From: 194952, To: 21651773
Edge ID: -30794402, From: 340702368, To: 180130885
Edge ID: -35398638, From: 415272936, To: 21651227
Edge ID: -35399697, From: 1647249339, To: 159896
Edge ID: -388487118#0, From: 5537722, To: 5537729
Edge ID: -388487118#11, From: 1148955258, To: 5537722
Edge ID: -388487118#12, From: 8082092, To: 1148955258
Edge ID: -388487118#13, From: 1148955258, To: 8082092
Edge ID: -388628848#2, From: 5711993, To:

# West to East


In [2]:
# Define the source and destination edges
source_edge = "151884975#0"
destination_edge = "151884974#0"

west_source_speed = net.getEdge(source_edge).getSpeed()  # Get the speed limit of the edge
# Find the shortest path between the edges
shortest_path = net.getShortestPath(net.getEdge(source_edge), net.getEdge(destination_edge))
print(shortest_path)

# Calculate the travel time
travel_time = 0
edges = list()
for edge in shortest_path[0]:
    edge_length = edge.getLength()  # Length of the edge in meters
    edge_speed = edge.getSpeed()   # Speed limit of the edge in m/s
    travel_time += edge_length / edge_speed  # Time = Distance / Speed
    edges.append(edge.getID())
W2E_route = ' '.join(edges)

print(f"Shortest path from {source_edge} to {destination_edge}: {W2E_route}")
print(f"Travel time from {source_edge} to {destination_edge}: {travel_time:.2f} seconds")

((<edge id="151884975#0" from="1647249313" to="1647249339"/>, <edge id="1080999537#0" from="1647249339" to="1647249369"/>, <edge id="151884977#0" from="1647249369" to="1647249378"/>, <edge id="151884977#4" from="1647249378" to="1647249387"/>, <edge id="151884974#0" from="1647249387" to="1639252584"/>), 949.72)
Shortest path from 151884975#0 to 151884974#0: 151884975#0 1080999537#0 151884977#0 151884977#4 151884974#0
Travel time from 151884975#0 to 151884974#0: 79.67 seconds


In [ ]:
west_source_speed

13.89

# East to West

In [4]:
# Define the source and destination edges
source_edge = "24225358#0"
destination_edge = "1243253622#0"
east_source_speed = net.getEdge(source_edge).getSpeed()  # Get the speed limit of the edge

# Find the shortest path between the edges
shortest_path = net.getShortestPath(net.getEdge(source_edge), net.getEdge(destination_edge))
print(shortest_path)
edges = list()

# Calculate the travel time
travel_time = 0
for edge in shortest_path[0]:
    edge_length = edge.getLength()  # Length of the edge in meters
    edge_speed = edge.getSpeed()   # Speed limit of the edge in m/s
    travel_time += edge_length / edge_speed  # Time = Distance / Speed
    edges.append(edge.getID())
E2W_route = ' '.join(edges)
print(f"Shortest path from {source_edge} to {destination_edge}: {E2W_route}")

print(f"Travel time from {source_edge} to {destination_edge}: {travel_time:.2f} seconds")

((<edge id="24225358#0" from="8082102" to="263066973"/>, <edge id="1285834640" from="263066973" to="1647249380"/>, <edge id="110107986#2" from="1647249380" to="194965"/>, <edge id="1243253630#0" from="194965" to="159896"/>, <edge id="98438064#0" from="159896" to="11554553903"/>, <edge id="1243253622#0" from="11554553903" to="159868"/>), 950.41)
Shortest path from 24225358#0 to 1243253622#0: 24225358#0 1285834640 110107986#2 1243253630#0 98438064#0 1243253622#0
Travel time from 24225358#0 to 1243253622#0: 79.73 seconds


In [5]:
east_source_speed

8.33

# East to sensor

In [8]:
# Define the source and destination edges
source_edge = "24225358#0"
destination_edge = "24225358#0"

# Find the shortest path between the edges
shortest_path = net.getShortestPath(net.getEdge(source_edge), net.getEdge(destination_edge))
print(shortest_path)

# Calculate the travel time
travel_time = 0
for edge in shortest_path[0]:
    edge_length = edge.getLength()  # Length of the edge in meters
    edge_speed = edge.getSpeed()   # Speed limit of the edge in m/s
    travel_time += edge_length / edge_speed  # Time = Distance / Speed

print(f"Travel time from {source_edge} to {destination_edge}: {travel_time:.2f} seconds")

((<edge id="24225358#0" from="8082102" to="263066973"/>,), 235.25)
Travel time from 24225358#0 to 24225358#0: 28.24 seconds


# West to Sensor

In [9]:
# Define the source and destination edges
source_edge = "151884975#0"
destination_edge = "151884977#4"

# Find the shortest path between the edges
shortest_path = net.getShortestPath(net.getEdge(source_edge), net.getEdge(destination_edge))
print(shortest_path)

# Calculate the travel time
travel_time = 0
for edge in shortest_path[0]:
    edge_length = edge.getLength()  # Length of the edge in meters
    edge_speed = edge.getSpeed()   # Speed limit of the edge in m/s
    travel_time += edge_length / edge_speed  # Time = Distance / Speed

print(f"Travel time from {source_edge} to {destination_edge}: {travel_time:.2f} seconds")

((<edge id="151884975#0" from="1647249313" to="1647249339"/>, <edge id="1080999537#0" from="1647249339" to="1647249369"/>, <edge id="151884977#0" from="1647249369" to="1647249378"/>, <edge id="151884977#4" from="1647249378" to="1647249387"/>), 714.56)
Travel time from 151884975#0 to 151884977#4: 51.44 seconds


In [10]:
trips.head()

,id,detector_id,timestamp,unix_epoch_time,depart,from,to,departLane
0,0,e2w_in,2025-05-09 22:18:00,1746829080,1746829080,151884975#0,151884974#0,1
1,1,w2e_in,2025-05-09 22:18:00,1746829080,1746829080,151884975#0,151884974#0,1
2,2,e2w_out,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,0
3,3,e2w_in,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,1
4,4,w2e_in,2025-05-09 22:18:21,1746829101,1746829101,151884975#0,151884974#0,1


In [11]:
routes_dict = dict()
routes_dict["w2e_out"] = W2E_route
routes_dict["w2e_in"] = W2E_route
routes_dict["e2w_out"] = E2W_route
routes_dict["e2w_in"] = E2W_route

def route_creating(trips,routes_dict):
    routes = trips.copy()
    for index, row in routes.iterrows():
        def convert_row(row):
            return """
            <vehicle id="%s" depart="%s" departLane="%s" departSpeed="0" departPos="0" arrivalPos="max">
                <route edges="%s"/>
            </vehicle>""" % (row.id, row.depart, row.departLane, routes_dict[row.detector_id])

        text0 = """<?xml version="1.0" encoding="UTF-8"?>\n\n\n"""
        text1 = """<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">"""
        text2 = '\n'.join(trips.apply(convert_row, axis=1))
        text3 = """\n</routes>"""
        with open("routes_sim.xml", 'w') as myfile: 
            myfile.write(text0+text1+text2+text3)

#vehid2pathid, all_routes_df = route_sampling(od_estimation, routes, hour)


In [12]:
route_creating(trips,routes_dict)